# langchain源码解读

## langchain模块

### langchain主要生态

- langchain 模块：以LCEL方式开发DAG结构的chain，也可以作为访问远程chain的python客户端
- langgraph 模块：让LCEL支持循环结构的chain
- langserve 模块：将chain转化为API
- langchain-js 模块：提供langchain的`js`版本，也可以作为访问远程chain的JS客户端
- langsmith 模块：跟踪和调试的平台
- langfuse 模块：langsmith的开源平替

### langchain模块的代码结构

- langchain：入口主模块
- core：核心模块
- community：社区贡献
- partners：合作伙伴模块
- experimetal：实验性模块
- cli：命令行

### LCEL相关类结构

- Runnable
    **实用方法:**
    - assign()
    - bind()
    - with_config()
    - get_graph()

    - RunnableSerializable（**LCEL**规则主要由**RunnableSequence**和**RunnableParallel**实现）
        **实用方法:**
        - dumps()
        - loads(json: str)

        **配置能力:**
        - RunnableBindingBase
            - RunnableBinding（向Runnable实例传递参数）
        - DynamicRunnable
            - RunnableConfigurableFields
            - RunnableConfigurableAlternatives

        **流程控制:**
        - RunnablePassthrough（传递额外输入）
        - RunnableSequence（实现顺序执行，可以用重载国的`|`符号或`RunnableSequence`来构造）
        - RunnableParallel（实现并行执行，可以用`Dict`或`RunnableParallel`类来构造，别名`RunnableMap`）

        **大模型:**
        - BaseLanguageModel
            - BaseLLM（派生其他大模型）
                - LLM（派生其他大模型）
                - OpenAIChat
                - Tongyi

        **提示语:**
        - BasePromptTemplate `[Dict, PromptValue]`
            - StringPromptTemplate（字符串模板）
            - BaseChatPromptTemplate（对话模板）
            - ImagePromptTemplate
            - PipelinePromptTemplate

        **检索器:**
        - BaseRetriever `[RetrieverInput, RetrieverOutput]`（派生各类检索器）

        **Tool:**
        - BaseTool `[Union[str, Dict], Any]`（派生各类工具）

        **输出解析:**      
        - BaseGenerationOutputParser `[Union[str, BaseMessage], T]`
        - BaseOutputParser（派生各类输出解析）

        **输入赋值:**      
        - RunnablePassthrough
        - RunnableAssign
        - RunnablePick

        **Chains:**      
        - Chain（结构化Runnable）
            - AgentExecutor（执行智能体）

    **定制能力:**
    - RunnableGenerator（常用于处理输出可能是迭代器结果的chain）
    - RunnableLambda（常用于包装普通函数，别名函数@chain）

## Runnable（提供原语的基类）

**LangChain Runnable** 是一种工作单元, 支持同步、异步、批处理和流式操作。

**LangChain表达式语言(LCEL)** 提供了一种声明式的方法来构建使用大语言模型的生产级程序。

使用 **LCEL** 和 **LangChain Runnable** 构建的程序内在地支持同步、异步、批处理和流式操作。

支持异步可以让托管基于 **LCEL** 程序的服务器更好地扩展,以处理更高的并发负载。

流式输出中间结果的能力可以创建更响应的用户体验。

**Runnable** 模块包含了 **LangChain Runnable** 原语的模式和实现。

### 最主要的原语方法

主要方法:

- invoke/ainvoke: 将单个输入转换为输出。  
- batch/abatch: 有效地将多个输入转换为输出。
- stream/astream: 流式输出单个输入生成的结果。
- astream_log: 流式输出输入的输出以及所选的中间结果。

内置优化:  

- 批处理: 默认情况下,批处理使用线程池执行器并行运行invoke()。 
  <br>可以重写以优化批处理。

- 异步: 带有“a”后缀的方法是异步的。默认情况下,它们使用asyncio的线程池执行同步版本。
  <br>可以重写为原生异步。

所有方法都接受一个可选的config参数,可用于配置执行、添加标签和元数据,以进行跟踪、调试等。  <br/>
Runnable通过input_schema属性、output_schema属性和config_schema方法公开其输入、输出和配置的模式信息。

### 链的基本示例

LangChain表达式语言(LCEL)是一种声明式的方式来将Runnable组合成链。
这样构建的任何链都将自动支持同步、异步、批处理和流操作。
主要的组合原语是RunnableSequence和RunnableParallel。

RunnableSequence按顺序调用一系列runnable,一个runnable的输出作为下一个的输入。
可以使用 | 运算符构造,或者通过向RunnableSequence传入runnable列表。
RunnableParallel并发调用runnable,向每个都提供相同的输入。
可以在序列中使用字典字面量构造,或者通过向RunnableParallel传入字典来构造。

代码示例：

In [251]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, chain

@chain
def a(x):
    return(x + 1)

@chain
def b(x):
    return(x + 2)

@chain
def c(dict):
    return(dict["x"] + dict["y"])
    
sequence = a | b

print(sequence.invoke(1))
print(sequence.batch([1, 2, 3]))

parral = {"x": a, "y": b} | c
print(parral.invoke(1)) # (1+1) + (1+2) = 5

4
[4, 5, 6]
5


### LCEL提供的好处

LangChain Expression Language（LCEL）是一种声明式语言，可轻松组合不同的调用顺序构成 Chain。LCEL 自创立之初就被设计为能够支持将原型投入生产环境，**无需代码更改**，从最简单的“提示+LLM”链到最复杂的链（已有用户成功在生产环境中运行包含数百个步骤的 LCEL Chain）。

LCEL的一些亮点包括：

1. **流支持**：使用 LCEL 构建 Chain 时，你可以获得最佳的首个令牌时间（即从输出开始到首批输出生成的时间）。
对于某些 Chain，这意味着可以直接从LLM流式传输令牌到流输出解析器，从而以与 LLM 提供商输出原始令牌相同的速率获得解析后的、增量的输出。

2. **异步支持**：任何使用 LCEL 构建的链条都可以通过同步API（例如，在 Jupyter 笔记本中进行原型设计时）和异步 API（例如，在 LangServe 服务器中）调用。
这使得相同的代码可用于原型设计和生产环境，具有出色的性能，并能够在同一服务器中处理多个并发请求。

3. **优化的并行执行**：当你的 LCEL 链条有可以并行执行的步骤时（例如，从多个检索器中获取文档），我们会自动执行，无论是在同步还是异步接口中，以实现最小的延迟。

4. **重试和回退**：为 LCEL 链的任何部分配置重试和回退。这是使链在规模上更可靠的绝佳方式。
目前我们正在添加重试/回退的流媒体支持，因此你可以在不增加任何延迟成本的情况下获得增加的可靠性。

5. **访问中间结果**：对于更复杂的链条，访问在最终输出产生之前的中间步骤的结果通常非常有用。
这可以用于让最终用户知道正在发生一些事情，甚至仅用于调试链条。你可以流式传输中间结果，并且在每个LangServe服务器上都可用。

6. **输入和输出模式**：输入和输出模式为每个 LCEL 链提供了从链的结构推断出的 Pydantic 和 JSONSchema 模式。
这可以用于输入和输出的验证，是 LangServe 的一个组成部分。

7. **无缝LangSmith跟踪集成**：随着链条变得越来越复杂，理解每一步发生了什么变得越来越重要。
通过 LCEL，所有步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。

8. **无缝LangServe部署集成**：任何使用 LCEL 创建的链都可以轻松地使用 LangServe 进行部署。

原文：[https://python.langchain.com/docs/expression_language]()

#### 流支持

#### 异步支持

#### 优化的并行执行

#### 重试和回退

with_retry：

In [103]:
from langchain_core.runnables import RunnableLambda
import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

chain = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt = 10,
        wait_exponential_jitter = False
    )
)

print("---RunnableSequence")
print(type(chain.first))
print(type(chain.middle))
print(type(chain.last))
print("---Runnable")
print(chain.get_name())
print(chain.get_prompts())
print(chain.input_schema.schema()) # Show inferred input schema
print(chain.output_schema.schema()) # Show inferred output schema

print(chain.invoke(2)) # invoke the sequence (note the retry above!!)

---RunnableSequence
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'list'>
<class 'langchain_core.runnables.retry.RunnableRetry'>
---Runnable
RunnableSequence
[]
{'title': 'add_one_input', 'type': 'integer'}
{'title': 'buggy_double_output', 'type': 'integer'}
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
6


#### 访问中间结果

**set_debug**

随着链的变长,能够看到中间结果以调试和跟踪链是很有用的。<br>
您可以将全局调试标志设置为True,以为所有链启用调试输出:

In [ ]:
from langchain_core.globals import set_debug
set_debug(True)

chain.invoke(3)

**callbacks**：可以将现有或自定义回调传递给任何给定的链

In [ ]:
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_core.globals import set_debug
set_debug(False)

chain.invoke(
    3,
    config={'callbacks': [ConsoleCallbackHandler()]} # 这与设置debug为true的效果类似
)

**绘制LCEL执行结构**

In [223]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.graph_draw import draw

def add_one(x: int) -> int:
    return x + 1

chain = (
    RunnableLambda(add_one) | add_one
)

chain.get_graph().print_ascii()

 +---------------+   
 | add_one_input |   
 +---------------+   
          *          
          *          
          *          
+-----------------+  
| Lambda(add_one) |  
+-----------------+  
          *          
          *          
          *          
+-----------------+  
| Lambda(add_one) |  
+-----------------+  
          *          
          *          
          *          
+----------------+   
| add_one_output |   
+----------------+   


## 配置能力

`RunnableBinding` 可以被看作是一个"Runnable对象的装饰器"，它保留了 `Runnable` 的基本特性，即批处理、流处理和异步支持，同时添加了额外的功能。

任何继承自 `Runnable` 的类都可以绑定到一个 `RunnableBinding`。
`Runnable` 提供了一套标准的方法来创建 `RunnableBindings` 或 `RunnableBindings` 的子类（例如，`RunnableRetry`，`RunnableWithFallbacks`）以添加额外的功能。

这些方法包括：
- `bind`：绑定 kwargs 以在运行底层可运行对象时传递。
- `with_config`：绑定配置以在运行底层可运行对象时传递。
- `with_listeners`：将生命周期监听器绑定到底层可运行对象。
- `with_types`：覆盖底层可运行对象的输入和输出类型。
- `with_retry`：将重试策略绑定到底层可运行对象。
- `with_fallbacks`：将回退策略绑定到底层可运行对象。

### 配置能力相关类结构

- Runnable
    - RunnableSerializable
        - RunnableBindingBase
            - RunnableBinding（向Runnable底层传递参数，`.bind()`）
            - RunnableWithMessageHistory（支持对话历史）
            - RunnableRetry（支持重试，`.with_retry()`）
            - HubRunnable（访问`LangChain Hub`的实例）
            - OpenAIFunctionsRouter
        - DynamicRunnable（支持动态配置）
            - RunnableConfigurableFields
            - RunnableConfigurableAlternatives
        - RunnableWithFallbacks（支持报错回滚，`.with_fallbacks()`）
        - RunnableAssign（`.assign()`）

### assign

为字典对象或RunnableParallel对象赋值：

In [306]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    num = itemgetter("num"),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
)

print(runnable.invoke({"num": 1}))

{'passed': {'num': 1}, 'num': 1, 'extra': {'num': 1, 'mult': 3}}


### bind

使用`bind`：可以绑定 kwargs 以在运行底层可运行对象时传递。

#### 将 stop 传递给 OpenAI

In [256]:
# 创建一个可运行绑定，它在运行时调用 ChatModel，并传递额外的 kwarg `stop=['-']`。
from langchain_openai import ChatOpenAI
model = ChatOpenAI()
model.invoke('Say "Parrot-MAGIC"', stop=['-']) # 应返回 `Parrot`

# 通过 `bind` 方法（它返回一个新的 RunnableBinding）来简单地使用它
runnable_binding = model.bind(stop=['-'])
runnable_binding.invoke('Say "Parrot-MAGIC"') # 应返回 `Parrot`

AIMessage(content='Parrot')

#### 将 seed 传递给 OpenAI

In [262]:
model.bind(seed=42).invoke("你是什么模型?")

AIMessage(content='我是一个基于神经网络的模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我是由OpenAI开发的，用于自然语言处理和生成文本的任务。')

In [263]:
model.bind(seed=42).invoke("你是什么模型?")

AIMessage(content='我是一个基于神经网络的模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我是由OpenAI开发的，用于自然语言处理和生成文本的任务。')

In [265]:
model.bind(seed=43).invoke("你是什么模型?")

AIMessage(content='我是一个基于人工智能技术的对话系统模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我被训练来理解和生成自然语言，可以提供各种领域的信息和帮助回答问题。')

### 根据配置动态选择Runnable对象

## 流程控制

Runnable 可以支持包括顺序、分支、条件、迭代等多种控制方式，构建较复杂的有向无环图。

与直接使用python代码控制流程不同的是，下面这些流程控制手段仍然保持以Runnable返回，以便获得LCEL的诸多额外好处，如：Runnable的统一方法、序列化能力、重试能力、回滚能力、Langsmith追踪和Langserve的API集成等。

### 流程控制相关类结构

- Runnable
    - RunnableSerializable
        - RunnableSequence（实现顺序执行，可以用|符号或RunnableSequence来构造）
        - RunnableParallel（实现并行执行，可以用Dict或RunnableParallel类来构造，别名RunnableMap）
        - RunnableEach（实现相同的链迭代多次）
        - RunnableBranch（实现流程分支，根据条件选择不同的链执行）
        - RouterRunnable（实现流程分支，根据枚举值选择不同的链执行）

### RunnableSerializable

主要增加了序列化的方法，可以将Runnable以JSON字符串的方式保存或加载：

- loads / dumps （按JSON字符串）
- load / dumpd （按Dict类型）

#### 将Runnable保存为JSON

In [95]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableGenerator
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from typing import Iterator

model = ChatOpenAI()
chant = (
    ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
    | model
    | StrOutputParser()
)

In [96]:
from langchain_core.load import loads, dumps
json = dumps(chant)
print(json)

{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "ChatPromptTemplate"], "kwargs": {"input_variables": ["topic"], "messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "HumanMessagePromptTemplate"], "kwargs": {"prompt": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["topic"], "template": "Give me a 3 word chant about {topic}", "template_format": "f-string", "partial_variables": {}}}}}]}}, "middle": [{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}}], "last": {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output_parser", "StrOutputParser"], "kwargs": {}}, "name": null}}


#### 从JSON中恢复Runnable并调用

In [99]:
loads(json).invoke({"topic":"苹果"})

'"Apple, sweet success!"'

#### 根据配置动态选择Runnable对象

In [231]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.runnables import ConfigurableField, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
openai = ChatOpenAI()
llm = OpenAI()
output_parser = StrOutputParser()

configurable_model = llm.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="llm_openai", 
    chat_openai=openai
)

configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

In [192]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "llm_openai"}
)

'\n\n"Sweet, cold, yum!"'

In [193]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "chat_openai"}
)

'\n\n"Scoop, swirl, savor!"'

#### 查看有哪些动态选择

### RunnableSequence

RunnableSequence 是最最重要的流程控制组件。

另外，RunnableSequence 也增加了几个属性：
- first: 第一个Runnable组件
- middle: 中间的Runnable组件列表
- last: 最后一个Runnable组件

In [228]:
from langchain_core.runnables import RunnableLambda
import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

chain = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt = 10,
        wait_exponential_jitter = False
    )
)
chain.get_graph().print_ascii()

print("---RunnableSequence")
print(type(chain.first))
print(type(chain.middle))
print(type(chain.last))

    +---------------+    
    | add_one_input |    
    +---------------+    
            *            
            *            
            *            
  +-----------------+    
  | Lambda(add_one) |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| Lambda(buggy_double) | 
+----------------------+ 
            *            
            *            
            *            
+---------------------+  
| buggy_double_output |  
+---------------------+  
---RunnableSequence
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'list'>
<class 'langchain_core.runnables.retry.RunnableRetry'>


### RunnableParallel

一种情况是从同一个输入生成多个分支链：

In [227]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
sequence.get_graph().print_ascii()

await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

                    +---------------+                     
                    | add_one_input |                     
                    +---------------+                     
                            *                             
                            *                             
                            *                             
                  +-----------------+                     
                  | Lambda(add_one) |                     
                  +-----------------+                     
                            *                             
                            *                             
                            *                             
          +----------------------------------+            
          | Parallel<mul_two,mul_three>Input |            
          +----------------------------------+            
                  ***               ***                   
               ***                     ***              

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

### RunnableEach

如果需要让同一个链执行多次，使用 RunnableEach 非常方便。

In [224]:
from langchain_core.runnables.base import RunnableEach, chain

@chain
def myfunc(x):
    return(x * 2)

runnable_each = RunnableEach(bound = myfunc)
runnable_each.get_graph().print_ascii()

output = runnable_each.invoke(range(1, 5))
print(output)

 +--------------+  
 | myfunc_input |  
 +--------------+  
         *         
         *         
         *         
+----------------+ 
| Lambda(myfunc) | 
+----------------+ 
         *         
         *         
         *         
+---------------+  
| myfunc_output |  
+---------------+  
[2, 4, 6, 8]


### RunnableBranch

有时候希望实现类似 ifelse 的条件分支，使用 RunnableBrach 可以接受 (条件函数, runnable) 的元组对和一个默认分支。

In [225]:
from langchain_core.runnables import RunnableBranch, chain

@chain
def myfunc(x):
    return(x * 2)

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), myfunc),
    lambda x: "goodbye",
)
branch.get_graph().print_ascii()

print(branch.invoke("hello")) # "HELLO"
print(branch.invoke(3)) # 6
print(branch.invoke(None)) # "goodbye"

+-------------+  
| BranchInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Branch |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| BranchOutput | 
+--------------+ 
HELLO
6
goodbye


### RouterRunnable

条件分支中还有一种情况更为简单，适合使用RouterRunnable，可以比RunnableBranch简洁得多。
<br>但传递参数时必须以 RouterInput 类型传入，或使用等价的字典结构。

```
class RouterInput(TypedDict):
    key: str
    input: Any
```

In [226]:
from langchain_core.runnables import RouterRunnable, chain

@chain
def lower(x):
    return(x.lower())

@chain
def upper(x):
    return(x.upper())

router = RouterRunnable({"lower": lower, "upper": upper})
router.get_graph().print_ascii()

print(router.invoke({"key": "upper", "input": "Hello"}))
print(router.invoke({"key": "lower", "input": "Hello"}))

+---------------------+  
| RouterRunnableInput |  
+---------------------+  
            *            
            *            
            *            
   +----------------+    
   | RouterRunnable |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| RouterRunnableOutput | 
+----------------------+ 
HELLO
hello


## 大模型

### 大模型相关类结构

- Runnable
    - RunnableSerializable
        - BaseLanguageModel（大模型）
            - BaseChatModel `[LanguageModelInput, LanguageModelOutputVar]`
                - SimpleChatModel
                - ChatOpenAI
                    - AzureChatOpenAI
            - BaseLLM
                - BaseOpenAI
                    - OpenAI
                    - AzureOpenAI
                - HuggingFacePipeline
                - LLM
                    - FakeListLLM
                        - FakeStreamingListLLM
                    - HumanInputLLM
                    - HuggingFaceHub
                    - HuggingFaceEndpoint
                    - HuggingFaceTextGenInference
                    - QianfanLLMEndpoint
                    - BaichuanLLM
                - Tongyi
                - Ollama

### 定制大模型

## 提示语和消息

### 提示语和消息相关类结构

- PromptValue
    - StringPromptValue
    - ChatPromptValue
        - ChatPromptValueConcrete
    - ImagePromptValue

- BaseMessage
    - BaseMessageChunk
    - AIMessage
        - AIMessageChunk(AIMessage, BaseMessageChunk)
    - ChatMessage
        - ChatMessageChunk(ChatMessage, BaseMessageChunk)
    - FunctionMessage
        - FunctionMessageChunk(FunctionMessage, BaseMessageChunk)
    - HumanMessage
        - HumanMessageChunk(HumanMessage, BaseMessageChunk)
    - SystemMessage
        - SystemMessageChunk(SystemMessage, BaseMessageChunk)
    - ToolMessage
        - ToolMessageChunk(ToolMessage, BaseMessageChunk)

- BaseExampleSelector
    - LengthBasedExampleSelector
    - SemanticSimilarityExampleSelector

- Runnable
    - RunnableSerializable
        - BasePromptTemplate `[Dict, PromptValue]`
            - StringPromptTemplate
                - PromptTemplate
                - FewShotPromptWithTemplates
                - FewShotPromptTemplate
            - BaseChatPromptTemplate
                - AutoGPTPrompt
                - ChatPromptTemplate
                    - AgentScratchPadChatPromptTemplate
            - ImagePromptTemplate
            - PipelinePromptTemplate
        - BaseMessagePromptTemplate
            - _StringImageMessagePromptTemplate
                - ChatMessagePromptTemplate
                - AIMessagePromptTemplate
                - HumanMessagePromptTemplate
                - SystemMessagePromptTemplate

### 提示语模板库

### 从文件加载提示语

### 相关类结构

In [ ]:
- BaseMessage

## 检索器

### 相关类结构

- Embeddings
    - embed_documents / aembed_documents
    - embed_query / aembed_query
        - OpenAIEmbeddings
            - AzureOpenAIEmbeddings

- Runnable
    - RunnableSerializable
        - BaseRetriever `[RetrieverInput, RetrieverOutput]`
            - VectorStoreRetriever
            - AzureCognitiveSearchRetriever
            - AmazonKnowledgeBasesRetriever
            - ChatGPTPluginRetriever
            - ElasticSearchBM25Retriever
            - KNNRetriever
            - LlamaIndexRetriever
            - MetalRetriever
            - MilvusRetriever
            - OutlineRetriever
            - PineconeHybridSearchRetriever
            - QdrantSparseVectorRetriever
            - RemoteLangChainRetriever
            - SVMRetriever
            - TavilySearchAPIRetriever
            - TFIDFRetriever
            - WeaviateHybridSearchRetriever
            - WikipediaRetriever

- BaseStore
- VectorStore
    - VectorStoreRetriever

## 输出解析和文档

### 相关类结构

- Document
- BaseDocumentTransformer

- Generation
    - ChatGeneration
        - ChatGenerationChunk
- ChatResult
- LLMResult
- RunInfo

- Runnable
    - RunnableSerializable
        - BaseGenerationOutputParser `[Union[str, BaseMessage], T]`
            - OutputFunctionsParser
                - PydanticOutputFunctionsParser
                    - PydanticAttrOutputFunctionsParser
            - JsonOutputToolsParser
                - JsonOutputKeyToolsParser
                - PydanticToolsParser
        - BaseOutputParser
            - BaseTransformOutputParser
                - BaseCumulativeTransformOutputParser
                    - JsonOutputParser（别名SimpleJsonOutputParser）
                    - JsonOutputFunctionsParser
                        - JsonKeyOutputFunctionsParser
                - StrOutputParser
                - XMLOutputParser
                - ListOutputParser
                    - CommaSeparatedListOutputParser
                    - NumberedListOutputParser
                    - MarkdownListOutputParser
            - BooleanOutputParser
            - CombiningOutputParser
            - DatetimeOutputParser
            - EnumOutputParser
            - OutputFixingParser（使用LLM修复错误）
            - PandasDataFrameOutputParser
            - PydanticOutputParser
            - RegexDictParser
            - RegexParser
            - RetryOutputParser
            - RetryWithErrorOutputParser
            - StructuredOutputParser
            - YamlOutputParser

### 定制一个输出解析器

## 回调和跟踪

### 相关类结构

- CallbackManagerMixin
    - BaseCallbackManager
        - CallbackManager
            - CallbackManagerForChainGroup
        - AsyncCallbackManager
            - AsyncCallbackManagerForChainGroup
- RunManagerMixin
    - BaseRunManager
        - RunManager
            - ParentRunManager
                - CallbackManagerForChainRun(ParentRunManager, ChainManagerMixin)
                - CallbackManagerForToolRun(ParentRunManager, ToolManagerMixin)
                - CallbackManagerForRetrieverRun(ParentRunManager, RetrieverManagerMixin)
            - CallbackManagerForLLMRun(RunManager, LLMManagerMixin)
    - AsyncRunManager
        - AsyncParentRunManager
            - AsyncCallbackManagerForChainRun(AsyncParentRunManager, ChainManagerMixin)
            - AsyncCallbackManagerForToolRun(AsyncParentRunManager, ToolManagerMixin)
            - AsyncCallbackManagerForRetrieverRun(AsyncParentRunManager, RetrieverManagerMixin)
        - AsyncCallbackManagerForLLMRun(AsyncRunManager, LLMManagerMixin)
- BaseCallbackHandler(
        LLMManagerMixin,
        ChainManagerMixin,
        ToolManagerMixin,
        RetrieverManagerMixin,
        CallbackManagerMixin,
        RunManagerMixin,
    )
    - AsyncCallbackHandler
    - StdOutCallbackHandler
    - StreamingStdOutCallbackHandler
    - BaseTracer
        - EvaluatorCallbackHandler
        - LangChainTracerV1
        - LangChainTracer
        - LogStreamCallbackHandler
        - RootListenersTracer
        - RunCollectorCallbackHandler
        - FunctionCallbackHandler
            - ConsoleCallbackHandler
- RunLogPatch
    - RunLog

- TracerSessionV1Base
    - TracerSessionV1Create
    - TracerSessionV1
    - TracerSessionBase
        - TracerSession
- ExampleBase
    - ExampleCreate
    - Example
- ExampleUpdate
- DataType
- DatasetBase
    - DatasetCreate
    - Dataset
- RunTypeEnum
- BaseRun
    - LLMRun
    - ChainRun 
    - ToolRun
    - Run
- RunLikeDict
- RunBase
    - RunWithAnnotationQueueInfo
- FeedbackSourceBase
    - APIFeedbackSource
    - ModelFeedbackSource
- FeedbackSourceType
- FeedbackBase
    - FeedbackCreate
    - Feedback
- TracerSession
    - TracerSessionResult
- BaseMessageLike
- DatasetShareSchema
- AnnotationQueue

### 自定义一个callback函数

In [309]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
import time

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        time.sleep(1)
        print(f"My custom handler, token: {token}")

# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in a list with our custom handler
chat = ChatOpenAI(max_tokens=25, streaming=True, callbacks=[MyCustomHandler()])

chat.invoke([HumanMessage(content="Tell me a joke")])

My custom handler, token: 
My custom handler, token: Why
My custom handler, token:  don
My custom handler, token: 't
My custom handler, token:  scientists
My custom handler, token:  trust
My custom handler, token:  atoms
My custom handler, token: ?
My custom handler, token:  


My custom handler, token: Because
My custom handler, token:  they
My custom handler, token:  make
My custom handler, token:  up
My custom handler, token:  everything
My custom handler, token: !
My custom handler, token: 


AIMessage(content="Why don't scientists trust atoms? \n\nBecause they make up everything!")

### 自定义个一个异步回调函数

In [313]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
import asyncio

class MyCustomHandler(BaseCallbackHandler):
    async def on_llm_new_token(self, token: str, **kwargs) -> None:
        await asyncio.sleep(1)
        print(f"My custom handler, token: {token}")

# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in a list with our custom handler
chat = ChatOpenAI(max_tokens=25, streaming=True, callbacks=[MyCustomHandler()])

await chat.ainvoke([HumanMessage(content="Tell me a joke")])

My custom handler, token: 
My custom handler, token: Sure
My custom handler, token: ,
My custom handler, token:  here
My custom handler, token: 's
My custom handler, token:  a
My custom handler, token:  joke
My custom handler, token:  for
My custom handler, token:  you
My custom handler, token: :


My custom handler, token: Why
My custom handler, token:  don
My custom handler, token: 't
My custom handler, token:  scientists
My custom handler, token:  trust
My custom handler, token:  atoms
My custom handler, token: ?


My custom handler, token: Because
My custom handler, token:  they
My custom handler, token:  make
My custom handler, token:  up
My custom handler, token:  everything
My custom handler, token: !
My custom handler, token: 


AIMessage(content="Sure, here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!")

## 记忆

### 记忆体相关结构

- BaseMemory

## 智能体

### 智能体相关类结构

**Agent**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（**ReAct**的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用**search 工具**不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（**ReAct**，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（**ReAct**，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持**OpenAI Function Calling**）
    - OPENAI_MULTI_FUNCTIONS（支持**OpenAI Function Calling**，支持多函数调度）

**主要结构**
- AgentAction
    - AgentActionMessageLog

- AgentStep
- AgentFinish

**智能体类结构**
- BaseSingleActionAgent
    - RunnableAgent
    - LLMSingleActionAgent（__遗留__，__修改为__：create_***_agent）
    - XMLAgent（__遗留__，__修改为__：create_xml_agent）
    - Agent
        - ChatAgent（__遗留__，__修改为__：create_react_agent）
        - ConversationalAgent（__遗留__，__修改为__：create_react_agent）
        - ConversationalChatAgent（__遗留__，__修改为__：create_json_chat_agent）
        - StructuredChatAgent（__遗留__，__修改为__：create_structured_chat_agent）
        - ZeroShotAgent（__遗留__，__修改为__：create_react_agent）
        - ReActDocstoreAgent（__遗留__）
            - ReActTextWorldAgent（__遗留__）
        - SelfAskWithSearchAgent（__遗留__，__修改为__：create_self_ask_with_search）
    - OpenAIFunctionsAgent（__遗留__，__修改为__：create_openai_functions_agent）

- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__遗留__，__修改为__：create_openai_tools_agent）

- Runnable
    - RunnableSerializable
        - Chain
            - AgentExecutor（__NEW__）
                - MRKLChain（__遗留__）
                - ReActChain（__遗留__）
                - SelfAskWithSearchChain（__遗留__）
        - OpenAIAssistantRunnable

- DocstoreExplorer（__遗留__）

**Prompt**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate `[Dict, PromptValue]`
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - AgentScratchPadChatPromptTemplate

**OutputParser**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - AgentOutputParser
            - MultiActionAgentOutputParser

### Function Calling 和 智能体的关系

在概念上，OpenAI tools 和 OpenAI function calling 并不属于 Agent 范畴。

Agent通常指的是可以自主做出决策和采取行动的系统。而 function calling 更像是一种技术手段,用于触发外部函数的执行。
<br>
但是, 在大部分 Agent 实现中, function calling 机制被用来实现 Agent 的行为，用来触发外部工具和服务。
所以它是往往启用 Agent 行为的一个组成部分。
总的来说,function calling本身只是一种技术功能。

而 OpenAI tools 是 OpenAI function calling 的替代机制，其主要区别在于:

- tools 支持同时调用多个函数, 而 function calling 只能调用单个函数。
- tools 允许函数之间交互和组合使用, function calling 中的函数是相互独立的。
- tools 提供了更多控制流程的选项, 如条件语句、循环等。
- tools 是 OpenAI 最新推出的函数调用机制, 而 function calling 是早期的格式,现在被视为legacy。


使用智能体框架比直接使用 tools 或 function calling 有以下几个主要优势:

- 封装性更好,使用更简单。智能体框架提供了现成的接口和抽象,开发者不需要处理底层的 tools 或 function calling细节。
- 支持更复杂的逻辑。智能体框架内置了决策、推理、计划等更高级的能力,不仅是简单的函数触发。
- 可解释性和可靠性更好。智能体的行为过程可以通过日志展示,输出更可解释。并且可以配置规则避免无意义的循环。
- 支持工具和服务的统一集成。工具可以用标准化方式集成到智能体框架中,而不需要不同的接入方式。
- 可扩展性更好,自定义能力更强。智能体框架提供了扩展接口,可以插入自定义的组件。

总体上,相比直接的 tools 或 function calling,智能体框架提供了更高层次的封装与抽象,使用更简单,功能也更全面和强大。
<br>这是使用智能体框架的主要优势所在。

### Tool

**Tool**
- Runnable
    - RunnableSerializable
        - BaseTool `[Union[str, Dict], Any]`
            - Tool
            - ExceptionTool
        
            **输入:**
            - StructuredTool
            - HumanInputRun
        
            **git:**
            - GitHubAction
            - GitLabAction
        
            **json:**
            - JsonListKeysTool
            - JsonGetValueTool
        
            **记忆:**
            - Memorize
        
            **天气查询:**
            - OpenWeatherMapQueryRun
        
            **sleep函数:**
            - SleepTool
        
            **shell:**
            - ShellTool
        
            **sql:**
            - QuerySQLDataBaseTool
            - InfoSQLDatabaseTool
            - QuerySQLCheckerTool
        
            **搜索引擎:**
            - TavilySearchResults
            - TavilyAnswer
        
            **QA向量检索:**
            - VectorStoreQATool
            - VectorStoreQAWithSourcesTool
        
            **wiki:**
            - WikipediaQueryRun
        
            **文件:**
            - CopyFileTool
            - DeleteFileTool
            - FileSearchTool
            - ListDirectoryTool
            - MoveFileTool
            - ReadFileTool
            - WriteFileTool


### 智能体和工具箱


#### pbi智能体
- create_pbi_agent
- create_pbi_chat_agent

#### sql生成智能体
- create_spark_sql_agent
- create_sql_agent

#### 结构化输出智能体
- create_xml_agent


create_json_agent
- create_json_chat_agent
- create_structured_chat_agent

#### 向量检索智能体
- create_vectorstore_agent
- create_vectorstore_router_agent

#### ReAct智能体
- create_react_agent

#### 自搜索智能体
- create_self_ask_with_search_agent

#### 遗留方法
- load_agent（__遗留__）
- load_agent_from_config（__遗留__）
- initialize_agent（__遗留__，__修改为__：create_***_agent）


### 文件管理

#### Toolkit：FileManagementToolkit

```
from langchain.agent_toolkits import FileManagementToolkit
```

包含的Tool清单：

- CopyFileTool
- DeleteFileTool
- FileSearchTool
- MoveFileTool
- ReadFileTool
- WriteFileTool
- ListDirectoryTool

### JSON查询

#### 创建Agent：create_json_agent

让智能体一边与JSON数据交互一边思考，将JSON查询结果作为答案返回。
```
from langchain.agents import create_json_agent
```

#### Toolkit：JsonToolkit

```
from langchain_community.agent_toolkits import JsonToolkit
```

包含的Tool清单：

- JsonListKeysTool
- JsonGetValueTool

#### 示例

In [428]:
import yaml
from langchain.agents import create_json_agent
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI

In [429]:
with open("data/openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0), toolkit=json_toolkit, verbose=True
)

In [443]:
json_agent_executor.invoke(
    "告诉我这个API的版本"
)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'paths', 'components']
Thought: I should look at the keys that exist in data["info"] to see what I have access to
Action: json_spec_list_keys
Action Input: data["info"]
Observation: ['title', 'description', 'version']
Thought: I should look at the value of data["info"]["version"] to see the API version
Action: json_spec_get_value
Action Input: data["info"]["version"]
Observation: v1
Thought: I now know the final answer
Final Answer: v1

> Finished chain.


{'input': '告诉我这个API的版本', 'output': 'v1'}

### OpenAI智能体

#### 创建Agent方法

**create_openapi_agent**

通过
  
**create_openai_functions_agent**

单一函数回调。

**create_openai_tools_agent**
  
多函数回调。

#### 环境准备

In [412]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent

In [413]:
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

#### 使用create_openai_functions_agent创建Agent

create_openai_functions_agent是openai早期实现的接口，仅支持一次函数调用：

In [415]:
functions_agent = create_openai_functions_agent(llm, tools, prompt)
functions_agent_executor = AgentExecutor(agent=functions_agent, tools=tools, verbose=True)
functions_agent_executor.invoke({
    "input": "Erick和Harrison都为langchain项目做了不少贡献，他们与langchain项目有什么关系吗?请用中文回答。"
})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Erick和Harrison与langchain项目的关系'}`


[{'url': 'https://zhuanlan.zhihu.com/p/645358531', 'content': '关于软件复杂性和复杂性下的流行性之争是永恒的话题。没有人愿意成为批评 LangChain 这样的免费开源软件的混蛋，但我愿意承担这个责任。明确地说，我并不反对 Harrison Chase 或 LangChain  简单来说，LangChain 是一个 Python 和 JavaScript 库，由 Harrison Chase 开发，用于连接 OpenAI 的 GPT API（后续已扩展到更多模型）以生成人工智能文本。  Chase 或 LangChain 的其他维护者（他们鼓励反馈）。  体现了「它很复杂，所以它一定更好」这一经常困扰后期代码库的哲学，可是 LangChain 甚至还不到一年。简单来说，LangChain 是一个 Python 和 JavaScript 库，由 Harrison Chase 开发，用于连接 OpenAI 的 GPT API（后续已扩展到更多模型）以生成人工智能文本。 更具体地说，它是论文《ReAct: Synergizing Reasoning and Acting in Language Models》的实现：该论文展示了一种提示技术，允许模型「推理」（通过思维链）和「行动」（通过能够使用预定义工具集中的工具，例如能够搜索互联网）。 论文链接： arxiv.org/pdf/2210.0362 事实证明，这种组合能够大幅提高输出文本的质量，并使大型语言模型具备正确解决问题的能力。'}]Erick和Harrison与langchain项目的关系是，Harrison Chase是LangChain项目的开发者，LangChain是一个Python和JavaScript库，用于连接OpenAI的GPT API以生成人工智能文本。因此，Erick和Harrison都与LangChain项目有密切的开发和贡献关系。

> Finished chain.


{'input': 'Erick和Harrison都为langchain项目做了不少贡献，他们与langchain项目有什么关系吗?请用中文回答。',
 'output': 'Erick和Harrison与langchain项目的关系是，Harrison Chase是LangChain项目的开发者，LangChain是一个Python和JavaScript库，用于连接OpenAI的GPT API以生成人工智能文本。因此，Erick和Harrison都与LangChain项目有密切的开发和贡献关系。'}

#### 使用create_openai_functions_agent创建Agent

使用create_openai_functions_agent时，“Erick和Harrison都为...”会被拆解为两个问题：

In [416]:
tools_agent = create_openai_tools_agent(llm, tools, prompt)
tools_agent_executor = AgentExecutor(agent=tools_agent, tools=tools, verbose=True)
tools_agent_executor.invoke({
    "input": "Erick和Harrison都为langchain项目做了不少贡献，他们与langchain项目有什么关系吗?请用中文回答。"
})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Erick langchain project'}`


[{'url': 'https://www.linkedin.com/pulse/llm-framework-how-langchain-redefine-application-2024-singh-ph-d--ikgmc', 'content': 'New to LinkedIn? Join now LLM Framework: How LangChain will Redefine Application Development in 2024  Embrace the power of LangChain and be part of this AI revolution.  In this ever-increasing need for Artificial Intelligence in all business forms, LangChain emerges as a game-changer,  This blog explores the multifaceted applications of LangChain and its groundbreaking impact on industries worldwide.LangChain is indeed a robust framework that is redefining AI application development. It allows developers to build context-aware reasoning applications with its flexible abstractions and AI-first ...'}]
Invoking: `tavily_search_results_json` with `{'query': 'Harrison langchain project'}`


[{'url': 'https://www.latent.space/p/langchain', 'co

{'input': 'Erick和Harrison都为langchain项目做了不少贡献，他们与langchain项目有什么关系吗?请用中文回答。',
 'output': 'Erick和Harrison都与LangChain项目有关。Erick在LangChain项目中探讨了LangChain的应用和影响，而Harrison则解释了LangChain是一个用于构建上下文感知推理应用的开源框架。因此，他们都与LangChain项目有密切关系。'}

#### 加入对话历史

In [417]:
from langchain_core.messages import AIMessage, HumanMessage
tools_agent_executor.invoke(
    {
        "input": "吴京的老婆主持过什么综艺节目?",
        "chat_history": [
            HumanMessage(content="吴京的老婆是谁？"),
            AIMessage(content="吴京的妻子是节目主持人谢楠。"),
        ],
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '谢楠主持过的综艺节目'}`


[{'url': 'https://ent.qianlong.com/2024/0207/8200732.shtml', 'content': '2024央视龙年春晚节目单 2024央视龙年春晚节目单 2月6日晚,2024央视龙年春晚北京主会场主持阵容正式官宣发布任鲁豫C位 2024央视龙年春晚湖南长沙分会场主持人发布:何炅、张舒越 2024央视龙年春晚新疆喀什分会场主持人马跃  附2024央视龙年春晚完整节目单 顾问:张艺谋、张一一 总导演:于蕾 主持人:任鲁豫、撒贝宁、尼格买提、龙洋、马凡舒(北京主会场) 张舒悦、何炅(湖南长沙分会场) 杨帆、刘心悦(辽宁沈阳分会场) 朱迅、徐杰(陕西西安分会场)  附2024央视龙年春晚完整节目单 顾问:张艺谋、张一一 总导演:于蕾 主持人:任鲁豫、撒贝宁、尼格买提、龙洋、马凡舒(北京主会场) 张舒悦、何炅(湖南长沙分会场) 杨帆、刘心悦(辽宁沈阳分会场) 朱迅、徐杰(陕西西安分会场)  综艺频道主持人杨帆与在2023中央广播电视总台主持人大赛中表现不俗获得银奖的辽宁卫视当家花旦刘心悦主持沈阳分会场,2018年和2020年已两次主持春晚分会场(贵州黔东南、河南郑州)的马跃和新疆卫视主持人维妮娜主持新疆喀什分会场,朱迅则与陕西2月6日,甲辰龙年2024中央广播电视总台春节联欢晚会进入最后3天的冲刺倒计时,随着龙年春晚四次带妆彩排的完成,通过现场观众、明星嘉宾、艺人助理 ...'}]根据搜索结果，谢楠曾主持过央视的春晚节目。您可以在以下链接中找到更多信息：[谢楠主持的综艺节目](https://ent.qianlong.com/2024/0207/8200732.shtml)

> Finished chain.


{'input': '吴京的老婆主持过什么综艺节目?',
 'chat_history': [HumanMessage(content='吴京的老婆是谁？'),
  AIMessage(content='吴京的妻子是节目主持人谢楠。')],
 'output': '根据搜索结果，谢楠曾主持过央视的春晚节目。您可以在以下链接中找到更多信息：[谢楠主持的综艺节目](https://ent.qianlong.com/2024/0207/8200732.shtml)'}

### create_csv_agent

### create_json_agent

### Toolkits

### 定制工具

### 定制智能体

## 其他

### 相关类结构

- BaseCache
- BaseChatMessageHistory
- ChatSession
- LangChainException
    - TracerException
    - OutputParserException

## Chains

### 在LCEL出现之前的遗留Chain

- Runnable
    - RunnableSerializable
        - Chain
            - LLMChain
                - ConversationChain
                - QuestionGeneratorChain
                - FlareChain
            - LLMCheckerChain
            - LLMRequestsChain
            - LLMMathChain（使用python代码执行数学计算）
            - LLMSummarizationCheckerChain
            - MapReduceChain
            - OpenAIModerationChain
            - SequentialChain
            - SimpleSequentialChain
            - APIChain
            - BaseCombineDocumentsChain
            - AnalyzeDocumentChain
            - ConstitutionalChain
            - BaseConversationalRetrievalChain
                - ConversationalRetrievalChain
                - ChatVectorDBChain
            - ElasticsearchDatabaseChain
            - NatBotChain（实现一个基于LLM的浏览器）
            - QAGenerationChain（问答对生成）
            - BaseQAWithSourcesChain
                - QAWithSourcesChain
            - BaseRetrievalQA
                - RetrievalQA
                - VectorDBQA
            - RouterChain
                - MultiRouteChain
                    - MultiRetrievalQAChain
                - EmbeddingRouterChain
                - LLMRouterChain

### 使用LCEL构建的预制 Chain

- create_stuff_documents_chain（**携带Document**的链）
- create_openai_fn_runnable（执行**OpenAI Function Calling**，在必要时选择一个函数执行）
- create_structured_output_runnable （执行**OpenAI Function Calling**，强制选择至少一个函数执行）
- load_query_constructor_runnable
- create_sql_query_chain（生成**SQL**的链）
- create_history_aware_retriever
- create_retrieval_chain（结合`create_stuff_documents_chain`构建**RAG**链）

#### create_stuff_documents_chain

In [314]:
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = create_stuff_documents_chain(llm, prompt)

docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

chain.invoke({"context": docs})

"Jesse's favorite color is red\nJamal's favorite color is orange"

#### create_openai_fn_runnable

In [315]:
from typing import Optional

from langchain.chains.openai_functions import create_openai_fn_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class RecordPerson(BaseModel):
    """Record some identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class RecordDog(BaseModel):
    """Record some identifying information about a dog."""

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")


llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for recording entities."),
        ("human", "Make calls to the relevant function to record the entities in the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)
chain = create_openai_fn_runnable([RecordPerson, RecordDog], llm, prompt)
chain.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

RecordDog(name='Harry', color='brown', fav_food='chicken')

#### load_query_constructor_runnable

#### create_sql_query_chain

In [328]:
# poetry add langchain langchain-community langchain-openai
# poetry add duckdb-engine
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 这段代码由于duckdb暂时不支持查询索引元数据，因此会出现警告信息
db = SQLDatabase.from_uri("duckdb:///data/langchain.duckdb")
llm = ChatOpenAI(temperature=0)
chain = create_sql_query_chain(llm, db)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/duckdb_engine/__init__.py:180: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [330]:
response = chain.invoke({"question": "我想知道文本块的长度分布，按照每1000个字一个步长分别统计"})
print(response)

SELECT (LENGTH(content) / 1000) * 1000 AS length_range, COUNT(*) AS count
FROM text_blocks
GROUP BY length_range
ORDER BY length_range
LIMIT 5;


#### create_history_aware_retriever

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import create_history_aware_retriever
from langchain import hub

rephrase_prompt = hub.pull("langchain-ai/chat-langchain-rephrase")
llm = ChatOpenAI()
retriever = ...
chat_retriever_chain = create_history_aware_retriever(
    llm, retriever, rephrase_prompt
)

chain.invoke({"input": "...", "chat_history": })

#### create_retrieval_chain

In [334]:
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")
documents = loader.load()

documents = CharacterTextSplitter().split_documents(documents)

embeddings = OpenAIEmbeddings()

In [344]:
import lancedb

db = lancedb.connect("data/demo.lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

search_docs = LanceDB.from_documents(documents, embeddings, connection=table)
retriever = search_docs.as_retriever()

In [ ]:
!poetry add langchainhub

In [350]:
search_docs.search(

In [348]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = ChatOpenAI()
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [349]:
retrieval_chain.invoke({"input": "总统说了什么？"})

{'input': '总统说了什么？',
 'context': [Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspire

## 支持的LLM

## 支持的向量数据库

## 项目模板

#### 输入和输出模式

#### 无缝LangSmith跟踪集成

#### 无缝LangServe部署集成